In [1]:
%load_ext dotenv
%dotenv 
# UPSTAGE_API_KEY from https://console.upstage.ai/

In [2]:
# @title set API key
import os
import getpass
from pprint import pprint
import warnings

warnings.filterwarnings("ignore")

from IPython import get_ipython

if "google.colab" in str(get_ipython()):
    # Running in Google Colab. Please set the UPSTAGE_API_KEY in the Colab Secrets
    from google.colab import userdata
    os.environ["UPSTAGE_API_KEY"] = userdata.get("UPSTAGE_API_KEY")
else:
    # Running locally. Please set the UPSTAGE_API_KEY in the .env file
    from dotenv import load_dotenv

    load_dotenv()

if "UPSTAGE_API_KEY" not in os.environ:
    os.environ["UPSTAGE_API_KEY"] = getpass.getpass("Enter your Upstage API key: ")


In [4]:
import os
import gradio as gr
from langchain_upstage import (
    ChatUpstage,
    UpstageEmbeddings,
    UpstageDocumentParseLoader,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS  
from langchain.schema import AIMessage, HumanMessage
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트 설정
def set_korean_font():
    if fm.findSystemFonts(fontpaths=None, fontext='ttf'):
        plt.rc('font', family='NanumGothic')
    else:
        print("한글 폰트를 찾을 수 없습니다. 영어 레이블을 사용합니다.")

set_korean_font()

# LLM 인스턴스 생성
def create_llm():
    return ChatUpstage()

# chat 함수 정의
def chat(message, history):
    # LLM 인스턴스를 생성하여 대화 수행
    llm = create_llm()
    response = llm(message)
    return response

# 문서 로딩 및 텍스트 분할
def load_and_split_docs(file_path):
    doc_loader = UpstageDocumentParseLoader(file_path, output_format='html')
    docs = doc_loader.load()
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    splits = text_splitter.split_documents(docs)
    return splits

# 문서 임베딩 및 FAISS 인덱싱
def embed_and_index_docs_with_faiss(splits, faiss_index_path="faiss_index"):
    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
    
    if os.path.exists(faiss_index_path):
        vectorstore = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)
    else:
        vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
        vectorstore.save_local(faiss_index_path)
    
    return vectorstore.as_retriever(search_kwargs={"k": 3})

# 투자 리포트 생성 함수
def generate_investment_report(goal, risk, duration, message):
    history = []

    # 투자 리포트 요청 메시지 생성
    combined_message = (
        f"투자 목표가 '{goal}'이고, 리스크 허용 범위가 '{risk}', 투자 기간이 {duration}년일 때 "
        f"주식, 채권, 현금 및 단기 채권의 비율을 각각 퍼센트로만 반환해 주세요. "
        f"예: 주식: 50%, 채권: 30%, 현금/단기 채권: 20%. 이후 설명은 별도의 단락에 작성해 주세요."
        f"추가 메시지 '{message}'에 대해서도 목표, 리스크, 기간을 고려한 맞춤형 투자 리포트를 제공해 주세요."
    )
    
    # LLM으로부터 응답 받기
    full_response = chat(combined_message, history)
    
    # 투자 리포트와 비율 분리
    allocation = {}
    report = ""
    parsing_allocation = True
    for line in full_response.split("\n"):
        if parsing_allocation and ":" in line and "%" in line:
            asset, percentage = line.split(":", 1)
            try:
                allocation[asset.strip()] = float(percentage.strip().replace("%", ""))
            except ValueError:
                report += line + "\n"
                parsing_allocation = False
        else:
            report += line + "\n"

    # 비율이 정확히 추출되지 않은 경우 기본 값 설정
    if not allocation:
        allocation = {"Stocks": 50, "Bonds": 30, "Cash & Short-Term Bonds": 20}
    
    # 전체 투자 리포트 생성
    full_report = (
        f"투자리포트:\n"
        f"주식: {allocation.get('Stocks', 0)}%, 채권: {allocation.get('Bonds', 0)}%, "
        f"현금/단기 채권: {allocation.get('Cash & Short-Term Bonds', 0)}%\n\n"
        f"{report.strip()}"
    )

    return full_report

# 매수/매도 추천 생성 함수
def generate_trade_recommendation(asset_name):
    history = []
    
    # 매수/매도 추천 요청 메시지 생성
    recommendation_message = (
        f"'{asset_name}'에 대한 매수 또는 매도 추천을 작성해 주세요. 이 추천은 자산의 현재 시장 상황, "
        f"재무 상태, 시장 동향 등을 고려하여 작성하고, 중립적인 입장에서 투자자가 고려해야 할 사항을 포함해 주세요."
    )
    
    # LLM으로부터 응답 받기
    recommendation = chat(recommendation_message, history)
    
    return recommendation

# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    gr.Markdown("# 나만의 애널리스트")
    gr.Markdown("투자 목표와 리스크 허용 범위를 입력하여 맞춤형 투자 리포트를 받고, 특정 자산에 대한 매수/매도 추천을 제공합니다.")

    with gr.Row():
        investment_goal = gr.Textbox(label="투자 목표", placeholder="예: 은퇴 자금 마련")
        risk_tolerance = gr.Radio(["낮음", "중간", "높음"], label="리스크 허용 범위", value="중간")
        investment_duration = gr.Slider(1, 30, value=5, label="투자 기간 (년)")
        user_message = gr.Textbox(label="추가 메시지", placeholder="특별히 알고 싶은 점을 입력하세요.")
    
    # 투자 리포트
    with gr.Row():
        investment_report_result = gr.Textbox(label="투자 리포트", lines=5, placeholder="여기에 투자 리포트가 표시됩니다.")
        submit_report_btn = gr.Button("투자 리포트 받기")
        submit_report_btn.click(
            generate_investment_report,
            inputs=[investment_goal, risk_tolerance, investment_duration, user_message],
            outputs=investment_report_result
        )

    # 매수/매도 추천
    with gr.Row():
        asset_name = gr.Textbox(label="자산 이름", placeholder="예: 삼성전자")
        trade_recommendation_result = gr.Textbox(label="매수/매도 추천", lines=5, placeholder="여기에 매수/매도 추천이 표시됩니다.")
        submit_trade_btn = gr.Button("매수/매도 추천 받기")
        submit_trade_btn.click(
            generate_trade_recommendation,
            inputs=asset_name,
            outputs=trade_recommendation_result
        )

if __name__ == "__main__":
    file_path = "hana.pdf"
    splits = load_and_split_docs(file_path)
    db_retriever = embed_and_index_docs_with_faiss(splits, "faiss_index")
    demo.launch()

* Running on local URL:  http://127.0.0.1:7887

To create a public link, set `share=True` in `launch()`.


/var/folders/y1/0bk4kswd619cd_rg_3yzf32r0000gn/T/ipykernel_61929/1098130443.py:33: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(message)
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/blocks.py", line 2018, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  Fil

In [5]:
import os
import gradio as gr
from langchain_upstage import (
    ChatUpstage,
    UpstageEmbeddings,
    UpstageDocumentParseLoader,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS  
from langchain.schema import AIMessage, HumanMessage
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트 설정
def set_korean_font():
    if fm.findSystemFonts(fontpaths=None, fontext='ttf'):
        plt.rc('font', family='NanumGothic')
    else:
        print("한글 폰트를 찾을 수 없습니다. 영어 레이블을 사용합니다.")

set_korean_font()

# LLM 인스턴스 생성
def create_llm():
    return ChatUpstage()

# chat 함수 정의
def chat(message, history):
    llm = create_llm()
    response = llm(message)
    return response.content if hasattr(response, 'content') else str(response)

# 문서 로딩 및 텍스트 분할
def load_and_split_docs(file_path):
    doc_loader = UpstageDocumentParseLoader(file_path, output_format='html')
    docs = doc_loader.load()
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    splits = text_splitter.split_documents(docs)
    return splits

# 문서 임베딩 및 FAISS 인덱싱
def embed_and_index_docs_with_faiss(splits, faiss_index_path="faiss_index"):
    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
    
    if os.path.exists(faiss_index_path):
        vectorstore = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)
    else:
        vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
        vectorstore.save_local(faiss_index_path)
    
    return vectorstore.as_retriever(search_kwargs={"k": 3})

# 투자 리포트 생성 함수
def generate_investment_report(goal, risk, duration, message):
    history = []

    combined_message = (
        f"투자 목표가 '{goal}'이고, 리스크 허용 범위가 '{risk}', 투자 기간이 {duration}년일 때 "
        f"주식, 채권, 현금 및 단기 채권의 비율을 각각 퍼센트로만 반환해 주세요. "
        f"예: 주식: 50%, 채권: 30%, 현금/단기 채권: 20%. 이후 설명은 별도의 단락에 작성해 주세요."
        f"추가 메시지 '{message}'에 대해서도 목표, 리스크, 기간을 고려한 맞춤형 투자 리포트를 제공해 주세요."
    )
    
    full_response = chat(combined_message, history)
    
    allocation = {}
    report = ""
    parsing_allocation = True
    for line in full_response.split("\n"):
        if parsing_allocation and ":" in line and "%" in line:
            asset, percentage = line.split(":", 1)
            try:
                allocation[asset.strip()] = float(percentage.strip().replace("%", ""))
            except ValueError:
                report += line + "\n"
                parsing_allocation = False
        else:
            report += line + "\n"

    if not allocation:
        allocation = {"Stocks": 50, "Bonds": 30, "Cash & Short-Term Bonds": 20}
    
    full_report = (
        f"투자 리포트:\n"
        f"주식: {allocation.get('Stocks', 0)}%, 채권: {allocation.get('Bonds', 0)}%, "
        f"현금/단기 채권: {allocation.get('Cash & Short-Term Bonds', 0)}%\n\n"
        f"{report.strip()}"
    )

    return full_report

# 매수/매도 추천 생성 함수
def generate_trade_recommendation(asset_name, goal, risk, duration):
    history = []
    
    recommendation_message = (
        f"'{asset_name}'에 대한 중립적인 매수 또는 매도 추천을 제공해 주세요. "
        f"투자 목표가 '{goal}'이고, 리스크 허용 범위가 '{risk}', 투자 기간이 {duration}년임을 고려하여, "
        f"이 자산이 해당 목표와 리스크에 적합한지 평가하고, 주요 사항을 간결하게 제공해 주세요."
    )
    
    recommendation = chat(recommendation_message, history)
    
    # 깔끔한 추천 내용만 반환
    return recommendation.strip()

# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    gr.Markdown("# 나만의 애널리스트")
    gr.Markdown("투자 목표와 리스크 허용 범위를 입력하여 맞춤형 투자 리포트를 받고, 특정 자산에 대한 매수/매도 추천을 제공합니다.")

    with gr.Row():
        investment_goal = gr.Textbox(label="투자 목표", placeholder="예: 은퇴 자금 마련")
        risk_tolerance = gr.Radio(["낮음", "중간", "높음"], label="리스크 허용 범위", value="중간")
        investment_duration = gr.Slider(1, 30, value=5, label="투자 기간 (년)")
        user_message = gr.Textbox(label="추가 메시지", placeholder="특별히 알고 싶은 점을 입력하세요.")
    
    # 투자 리포트
    with gr.Row():
        investment_report_result = gr.Textbox(label="투자 리포트", lines=5, placeholder="여기에 투자 리포트가 표시됩니다.")
        submit_report_btn = gr.Button("투자 리포트 받기")
        submit_report_btn.click(
            generate_investment_report,
            inputs=[investment_goal, risk_tolerance, investment_duration, user_message],
            outputs=investment_report_result
        )

    # 매수/매도 추천
    with gr.Row():
        asset_name = gr.Textbox(label="자산 이름", placeholder="예: 삼성전자")
        trade_recommendation_result = gr.Textbox(label="매수/매도 추천", lines=5, placeholder="여기에 매수/매도 추천이 표시됩니다.")
        submit_trade_btn = gr.Button("매수/매도 추천 받기")
        submit_trade_btn.click(
            generate_trade_recommendation,
            inputs=[asset_name, investment_goal, risk_tolerance, investment_duration],
            outputs=trade_recommendation_result
        )

if __name__ == "__main__":
    file_path = "hana.pdf"
    splits = load_and_split_docs(file_path)
    db_retriever = embed_and_index_docs_with_faiss(splits, "faiss_index")
    demo.launch()

* Running on local URL:  http://127.0.0.1:7888

To create a public link, set `share=True` in `launch()`.


In [6]:
import os
import gradio as gr
from langchain_upstage import (
    ChatUpstage,
    UpstageEmbeddings,
    UpstageDocumentParseLoader,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS  
from langchain.schema import AIMessage, HumanMessage
import re

# 한글 폰트 설정
def set_korean_font():
    if fm.findSystemFonts(fontpaths=None, fontext='ttf'):
        plt.rc('font', family='NanumGothic')
    else:
        print("한글 폰트를 찾을 수 없습니다. 영어 레이블을 사용합니다.")

set_korean_font()

# LLM 인스턴스 생성
def create_llm():
    return ChatUpstage()

# chat 함수 정의
def chat(message, history):
    llm = create_llm()
    response = llm(message)
    return response.content if hasattr(response, 'content') else str(response)

# 기업명 추출 함수
def extract_company_name(user_input):
    match = re.match(r"'?(.+?)'?\s*살까", user_input)
    return match.group(1) if match else None

# FAISS 검색을 이용하여 관련 정보 추출 함수
def retrieve_related_info(company_name):
    if db_retriever:
        search_results = db_retriever(company_name)
        return "\n".join([doc.page_content for doc in search_results])
    return "관련 정보를 찾을 수 없습니다."

# 기업 분석 생성 함수
def generate_company_analysis(company_name, goal, risk, duration):
    history = []
    related_info = retrieve_related_info(company_name)

    analysis_message = (
        f"'{company_name}'에 대한 기업 분석을 제공해 주세요. "
        f"투자 목표가 '{goal}', 리스크 허용 범위가 '{risk}', 투자 기간이 {duration}년임을 고려하여, "
        f"다음 관련 정보도 포함해 분석을 제공해 주세요:\n{related_info}"
    )
    analysis = chat(analysis_message, history)
    return analysis.strip()

# 매수/매도 추천 생성 함수
def generate_trade_recommendation(company_name, goal, risk, duration):
    history = []
    related_info = retrieve_related_info(company_name)
    
    recommendation_message = (
        f"'{company_name}'에 대한 중립적인 매수 또는 매도 추천을 제공해 주세요. "
        f"투자 목표가 '{goal}'이고, 리스크 허용 범위가 '{risk}', 투자 기간이 {duration}년임을 고려하여, "
        f"다음 관련 정보도 참고하여 추천을 제공해 주세요:\n{related_info}"
    )
    
    recommendation = chat(recommendation_message, history)
    return recommendation.strip()

# 통합 함수: 기업 분석과 매수/매도 추천 출력
def generate_analysis_and_recommendation(user_input, goal, risk, duration):
    company_name = extract_company_name(user_input)
    if not company_name:
        return "올바른 입력 형식이 아닙니다. '기업명 살까?' 형식으로 입력해 주세요.", ""
    
    # 기업 분석 및 매수/매도 추천 생성
    analysis = generate_company_analysis(company_name, goal, risk, duration)
    recommendation = generate_trade_recommendation(company_name, goal, risk, duration)
    
    return analysis, recommendation

# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    gr.Markdown("# 나만의 애널리스트")
    gr.Markdown("투자 목표와 리스크 허용 범위를 입력하고 '기업명 살까?'와 같이 입력하면 해당 기업의 분석과 매수/매도 추천을 제공합니다.")

    with gr.Row():
        investment_goal = gr.Textbox(label="투자 목표", placeholder="예: 은퇴 자금 마련")
        risk_tolerance = gr.Radio(["낮음", "중간", "높음"], label="리스크 허용 범위", value="중간")
        investment_duration = gr.Slider(1, 30, value=5, label="투자 기간 (년)")
        user_input = gr.Textbox(label="질문 입력", placeholder="예: 삼성전자 살까?")
    
    # 기업 분석과 매수/매도 추천 결과 표시
    with gr.Row():
        company_analysis_result = gr.Textbox(label="기업 분석", lines=5, placeholder="여기에 기업 분석이 표시됩니다.")
        trade_recommendation_result = gr.Textbox(label="매수/매도 추천", lines=5, placeholder="여기에 매수/매도 추천이 표시됩니다.")
    
    # 버튼 설정 및 함수 연결
    submit_btn = gr.Button("분석 및 추천 받기")
    submit_btn.click(
        generate_analysis_and_recommendation,
        inputs=[user_input, investment_goal, risk_tolerance, investment_duration],
        outputs=[company_analysis_result, trade_recommendation_result]
    )

if __name__ == "__main__":
    file_path = "hana.pdf"
    splits = load_and_split_docs(file_path)
    db_retriever = embed_and_index_docs_with_faiss(splits, "faiss_index")
    demo.launch()

* Running on local URL:  http://127.0.0.1:7889

To create a public link, set `share=True` in `launch()`.


In [8]:
import os
import gradio as gr
from langchain_upstage import (
    ChatUpstage,
    UpstageEmbeddings,
    UpstageDocumentParseLoader,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS  
from langchain.schema import AIMessage, HumanMessage
import re

# 한글 폰트 설정
def set_korean_font():
    if fm.findSystemFonts(fontpaths=None, fontext='ttf'):
        plt.rc('font', family='NanumGothic')
    else:
        print("한글 폰트를 찾을 수 없습니다. 영어 레이블을 사용합니다.")

set_korean_font()

# LLM 인스턴스 생성
def create_llm():
    return ChatUpstage()

# chat 함수 정의
def chat(message, history):
    llm = create_llm()
    response = llm(message)
    return response.content if hasattr(response, 'content') else str(response)

# 기업명 추출 함수
def extract_company_name(user_input):
    match = re.match(r"'?(.+?)'?\s*살까", user_input)
    return match.group(1) if match else None

# FAISS 검색을 이용하여 관련 정보 추출 함수
def retrieve_related_info(company_name):
    if db_retriever:
        search_results = db_retriever.get_relevant_documents(company_name)
        return "\n".join([doc.page_content for doc in search_results])
    return "관련 정보를 찾을 수 없습니다."

# 기업 분석 생성 함수
def generate_company_analysis(company_name, goal, risk, duration):
    history = []
    related_info = retrieve_related_info(company_name)

    analysis_message = (
        f"'{company_name}'에 대한 기업 분석을 제공해 주세요. "
        f"투자 목표가 '{goal}', 리스크 허용 범위가 '{risk}', 투자 기간이 {duration}년임을 고려하여, "
        f"다음 관련 정보도 포함해 분석을 제공해 주세요:\n{related_info}"
    )
    analysis = chat(analysis_message, history)
    return analysis.strip()

# 매수/매도 추천 생성 함수
def generate_trade_recommendation(company_name, goal, risk, duration):
    history = []
    related_info = retrieve_related_info(company_name)
    
    recommendation_message = (
        f"'{company_name}'에 대한 중립적인 매수 또는 매도 추천을 제공해 주세요. "
        f"투자 목표가 '{goal}'이고, 리스크 허용 범위가 '{risk}', 투자 기간이 {duration}년임을 고려하여, "
        f"다음 관련 정보도 참고하여 추천을 제공해 주세요:\n{related_info}"
    )
    
    recommendation = chat(recommendation_message, history)
    return recommendation.strip()

# 통합 함수: 기업 분석과 매수/매도 추천 출력
def generate_analysis_and_recommendation(user_input, goal, risk, duration):
    company_name = extract_company_name(user_input)
    if not company_name:
        return "올바른 입력 형식이 아닙니다. '기업명 살까?' 형식으로 입력해 주세요.", ""
    
    # 기업 분석 및 매수/매도 추천 생성
    analysis = generate_company_analysis(company_name, goal, risk, duration)
    recommendation = generate_trade_recommendation(company_name, goal, risk, duration)
    
    return analysis, recommendation

# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    gr.Markdown("# 관심 종목 매수/매도 추천")
    gr.Markdown("투자 목표와 리스크 허용 범위를 입력하고 '기업명'을 입력하면 해당 기업의 분석과 매수/매도 추천을 제공합니다.")

    with gr.Row():
        investment_goal = gr.Textbox(label="투자 목표", placeholder="예: 은퇴 자금 마련")
        risk_tolerance = gr.Radio(["낮음", "중간", "높음"], label="리스크 허용 범위", value="중간")
        investment_duration = gr.Slider(1, 30, value=5, label="투자 기간 (년)")
        user_input = gr.Textbox(label="질문 입력", placeholder="예: 삼성전자 살까?")
    
    # 기업 분석과 매수/매도 추천 결과 표시
    with gr.Row():
        company_analysis_result = gr.Textbox(label="기업 분석", lines=5, placeholder="여기에 기업 분석이 표시됩니다.")
        trade_recommendation_result = gr.Textbox(label="매수/매도 추천", lines=5, placeholder="여기에 매수/매도 추천이 표시됩니다.")
    
    # 버튼 설정 및 함수 연결
    submit_btn = gr.Button("분석 및 추천 받기")
    submit_btn.click(
        generate_analysis_and_recommendation,
        inputs=[user_input, investment_goal, risk_tolerance, investment_duration],
        outputs=[company_analysis_result, trade_recommendation_result]
    )

if __name__ == "__main__":
    file_path = "hana.pdf"
    splits = load_and_split_docs(file_path)
    db_retriever = embed_and_index_docs_with_faiss(splits, "faiss_index")
    demo.launch()

* Running on local URL:  http://127.0.0.1:7891

To create a public link, set `share=True` in `launch()`.


/var/folders/y1/0bk4kswd619cd_rg_3yzf32r0000gn/T/ipykernel_61929/2750563533.py:42: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  search_results = db_retriever.get_relevant_documents(company_name)


In [10]:
import os
import gradio as gr
from langchain_upstage import (
    ChatUpstage,
    UpstageEmbeddings,
    UpstageDocumentParseLoader,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS  
from langchain.schema import AIMessage, HumanMessage

# 한글 폰트 설정
def set_korean_font():
    if fm.findSystemFonts(fontpaths=None, fontext='ttf'):
        plt.rc('font', family='NanumGothic')
    else:
        print("한글 폰트를 찾을 수 없습니다. 영어 레이블을 사용합니다.")

set_korean_font()

# LLM 인스턴스 생성
def create_llm():
    return ChatUpstage()

# chat 함수 정의
def chat(message, history):
    llm = create_llm()
    response = llm(message)
    return response.content if hasattr(response, 'content') else str(response)

# FAISS 검색을 이용하여 관련 정보 추출 함수
def retrieve_related_info(company_name):
    if db_retriever:
        search_results = db_retriever.get_relevant_documents(company_name)
        return "\n".join([doc.page_content for doc in search_results])
    return "관련 정보를 찾을 수 없습니다."

# 기업 분석 생성 함수
def generate_company_analysis(company_name, goal, risk, duration):
    history = []
    related_info = retrieve_related_info(company_name)

    analysis_message = (
        f"'{company_name}'에 대한 기업 분석을 제공해 주세요. "
        f"투자 목표가 '{goal}', 리스크 허용 범위가 '{risk}', 투자 기간이 {duration}년임을 고려하여, "
        f"다음 관련 정보도 포함해 분석을 제공해 주세요:\n{related_info}"
    )
    analysis = chat(analysis_message, history)
    return analysis.strip()

# 매수/매도 추천 생성 함수
def generate_trade_recommendation(company_name, goal, risk, duration):
    history = []
    related_info = retrieve_related_info(company_name)
    
    recommendation_message = (
        f"'{company_name}'에 대한 중립적인 매수 또는 매도 추천을 제공해 주세요. "
        f"투자 목표가 '{goal}'이고, 리스크 허용 범위가 '{risk}', 투자 기간이 {duration}년임을 고려하여, "
        f"다음 관련 정보도 참고하여 추천을 제공해 주세요:\n{related_info}"
    )
    
    recommendation = chat(recommendation_message, history)
    return recommendation.strip()

# 통합 함수: 기업 분석과 매수/매도 추천 출력
def generate_analysis_and_recommendation(company_name, goal, risk, duration):
    if not company_name:
        return "기업명을 입력해 주세요.", ""
    
    # 기업 분석 및 매수/매도 추천 생성
    analysis = generate_company_analysis(company_name, goal, risk, duration)
    recommendation = generate_trade_recommendation(company_name, goal, risk, duration)
    
    return analysis, recommendation

# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    gr.Markdown("# 관심 종목 매수/매도 추천")
    gr.Markdown("투자 목표와 리스크 허용 범위를 입력하고 '기업명'을 입력하면 해당 기업의 분석과 매수/매도 추천을 제공합니다.")
    
    with gr.Row():
        company_name = gr.Textbox(label="기업명", placeholder="예: 삼성전자")
    
    with gr.Row():
        investment_goal = gr.Textbox(label="투자 목표", placeholder="예: 은퇴 자금 마련")
        risk_tolerance = gr.Radio(["낮음", "중간", "높음"], label="리스크 허용 범위", value="중간")
        investment_duration = gr.Slider(1, 30, value=5, label="투자 기간 (년)")
    
    # 버튼 설정 및 함수 연결
    submit_btn = gr.Button("분석 및 추천 받기")
    submit_btn.click(
        generate_analysis_and_recommendation,
        inputs=[company_name, investment_goal, risk_tolerance, investment_duration],
        outputs=[company_analysis_result, trade_recommendation_result]
    )

    # 기업 분석과 매수/매도 추천 결과 표시
    with gr.Row():
        company_analysis_result = gr.Textbox(label="기업 분석", lines=5, placeholder="여기에 기업 분석이 표시됩니다.")
        trade_recommendation_result = gr.Textbox(label="매수/매도 추천", lines=5, placeholder="여기에 매수/매도 추천이 표시됩니다.")

if __name__ == "__main__":
    file_path = "hana.pdf"
    splits = load_and_split_docs(file_path)
    db_retriever = embed_and_index_docs_with_faiss(splits, "faiss_index")
    demo.launch()

* Running on local URL:  http://127.0.0.1:7893

To create a public link, set `share=True` in `launch()`.


In [12]:
import os
import gradio as gr
from langchain_upstage import (
    ChatUpstage,
    UpstageEmbeddings,
    UpstageDocumentParseLoader,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS  
from langchain.schema import AIMessage, HumanMessage

# 한글 폰트 설정
def set_korean_font():
    if fm.findSystemFonts(fontpaths=None, fontext='ttf'):
        plt.rc('font', family='NanumGothic')
    else:
        print("한글 폰트를 찾을 수 없습니다. 영어 레이블을 사용합니다.")

set_korean_font()

# LLM 인스턴스 생성
def create_llm():
    return ChatUpstage()

# chat 함수 정의
def chat(message, history):
    llm = create_llm()
    response = llm(message)
    return response.content if hasattr(response, 'content') else str(response)

# FAISS 검색을 이용하여 관련 정보 추출 함수
def retrieve_related_info(company_name):
    if db_retriever:
        search_results = db_retriever.get_relevant_documents(company_name)
        return "\n".join([doc.page_content for doc in search_results])
    return "관련 정보를 찾을 수 없습니다."

# 기업 분석 생성 함수
def generate_company_analysis(company_name, goal, risk, duration):
    history = []
    related_info = retrieve_related_info(company_name)

    analysis_message = (
        f"'{company_name}'에 대한 기업 분석을 제공해 주세요. "
        f"투자 목표가 '{goal}', 리스크 허용 범위가 '{risk}', 투자 기간이 {duration}년임을 고려하여, "
        f"다음 관련 정보도 포함해 분석을 제공해 주세요:\n{related_info}"
    )
    analysis = chat(analysis_message, history)
    return analysis.strip()

# 매수/매도 추천 생성 함수
def generate_trade_recommendation(company_name, goal, risk, duration):
    history = []
    related_info = retrieve_related_info(company_name)
    
    recommendation_message = (
        f"'{company_name}'에 대한 중립적인 매수 또는 매도 추천을 제공해 주세요. "
        f"투자 목표가 '{goal}'이고, 리스크 허용 범위가 '{risk}', 투자 기간이 {duration}년임을 고려하여, "
        f"다음 관련 정보도 참고하여 추천을 제공해 주세요:\n{related_info}"
    )
    
    recommendation = chat(recommendation_message, history)
    return recommendation.strip()

# 최종 매수/매도 결정 생성 함수
def generate_final_decision(recommendation_text):
    history = []
    
    final_decision_message = (
        f"추천 분석을 통해 최종적으로 매수 또는 매도 중 어떤 선택이 적합한지 간단히 결정해 주세요:\n{recommendation_text}"
    )
    final_decision = chat(final_decision_message, history)
    return final_decision.strip()

# 통합 함수: 기업 분석과 매수/매도 추천 및 최종 결론 출력
def generate_analysis_recommendation_decision(company_name, goal, risk, duration):
    if not company_name:
        return "기업명을 입력해 주세요.", "", ""
    
    # 기업 분석 및 매수/매도 추천 생성
    analysis = generate_company_analysis(company_name, goal, risk, duration)
    recommendation = generate_trade_recommendation(company_name, goal, risk, duration)
    
    # 최종 매수/매도 결론 생성
    final_decision = generate_final_decision(recommendation)
    
    return analysis, recommendation, final_decision

# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    gr.Markdown("# 관심 종목 매수/매도 추천")
    gr.Markdown("투자 목표와 리스크 허용 범위를 입력하고 '기업명'을 입력하면 해당 기업의 분석과 매수/매도 추천을 제공합니다.")
    
    with gr.Row():
        company_name = gr.Textbox(label="기업명", placeholder="예: 삼성전자")
    
    with gr.Row():
        investment_goal = gr.Textbox(label="투자 목표", placeholder="예: 은퇴 자금 마련")
        risk_tolerance = gr.Radio(["낮음", "중간", "높음"], label="리스크 허용 범위", value="중간")
        investment_duration = gr.Slider(1, 30, value=5, label="투자 기간 (년)")
    
    # 기업 분석과 매수/매도 추천 및 최종 결정 결과 표시를 위한 텍스트 박스 컴포넌트 추가
    company_analysis_result = gr.Textbox(label="기업 분석", lines=5, placeholder="여기에 기업 분석이 표시됩니다.")
    trade_recommendation_result = gr.Textbox(label="매수/매도 추천", lines=5, placeholder="여기에 매수/매도 추천이 표시됩니다.")
    final_decision_result = gr.Textbox(label="최종 결정", lines=2, placeholder="최종 매수/매도 결정을 여기에 표시합니다.")

    # 버튼 설정 및 함수 연결
    submit_btn = gr.Button("분석 및 추천 받기")
    submit_btn.click(
        generate_analysis_recommendation_decision,
        inputs=[company_name, investment_goal, risk_tolerance, investment_duration],
        outputs=[company_analysis_result, trade_recommendation_result, final_decision_result]
    )

if __name__ == "__main__":
    file_path = "hana.pdf"
    splits = load_and_split_docs(file_path)
    db_retriever = embed_and_index_docs_with_faiss(splits, "faiss_index")
    demo.launch()

* Running on local URL:  http://127.0.0.1:7894

To create a public link, set `share=True` in `launch()`.


In [13]:
import os
import gradio as gr
from langchain_upstage import (
    ChatUpstage,
    UpstageEmbeddings,
    UpstageDocumentParseLoader,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS  
from langchain.schema import AIMessage, HumanMessage

# 한글 폰트 설정
def set_korean_font():
    if fm.findSystemFonts(fontpaths=None, fontext='ttf'):
        plt.rc('font', family='NanumGothic')
    else:
        print("한글 폰트를 찾을 수 없습니다. 영어 레이블을 사용합니다.")

set_korean_font()

# LLM 인스턴스 생성
def create_llm():
    return ChatUpstage()

# chat 함수 정의
def chat(message, history):
    llm = create_llm()
    response = llm(message)
    return response.content if hasattr(response, 'content') else str(response)

# FAISS 검색을 이용하여 관련 정보 추출 함수
def retrieve_related_info(company_name):
    if db_retriever:
        search_results = db_retriever.get_relevant_documents(company_name)
        return "\n".join([doc.page_content for doc in search_results])
    return "관련 정보를 찾을 수 없습니다."

# 기업 분석 생성 함수
def generate_company_analysis(company_name, goal, risk, duration):
    history = []
    related_info = retrieve_related_info(company_name)

    analysis_message = (
        f"'{company_name}'에 대한 기업 분석을 제공해 주세요. "
        f"투자 목표가 '{goal}', 리스크 허용 범위가 '{risk}', 투자 기간이 {duration}년임을 고려하여, "
        f"다음 관련 정보도 포함해 분석을 제공해 주세요:\n{related_info}"
    )
    analysis = chat(analysis_message, history)
    return analysis.strip()

# 매수/매도 추천 생성 함수
def generate_trade_recommendation(company_name, goal, risk, duration):
    history = []
    related_info = retrieve_related_info(company_name)
    
    recommendation_message = (
        f"'{company_name}'에 대한 중립적인 매수 또는 매도 추천을 제공해 주세요. "
        f"투자 목표가 '{goal}'이고, 리스크 허용 범위가 '{risk}', 투자 기간이 {duration}년임을 고려하여, "
        f"다음 관련 정보도 참고하여 추천을 제공해 주세요:\n{related_info}"
    )
    
    recommendation = chat(recommendation_message, history)
    return recommendation.strip()

# 최종 매수/매도 결정 생성 함수
def generate_final_decision(recommendation_text):
    history = []
    
    final_decision_message = (
        f"다음 추천 분석을 바탕으로 최종적으로 매수 또는 매도 중 하나를 선택하여 결정해 주세요:\n{recommendation_text}"
    )
    final_decision = chat(final_decision_message, history)
    
    # 결과를 간단히 "매수" 또는 "매도"만 반환하도록 수정
    if "매수" in final_decision:
        return "매수"
    elif "매도" in final_decision or "중립" in final_decision or "hold" in final_decision.lower():
        return "매도"
    else:
        return "추천 없음"  # 매수 또는 매도가 명시되지 않은 경우

# 통합 함수: 기업 분석과 매수/매도 추천 및 최종 결론 출력
def generate_analysis_recommendation_decision(company_name, goal, risk, duration):
    if not company_name:
        return "기업명을 입력해 주세요.", "", ""
    
    # 기업 분석 및 매수/매도 추천 생성
    analysis = generate_company_analysis(company_name, goal, risk, duration)
    recommendation = generate_trade_recommendation(company_name, goal, risk, duration)
    
    # 최종 매수/매도 결론 생성
    final_decision = generate_final_decision(recommendation)
    
    return analysis, recommendation, final_decision

# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    gr.Markdown("# 관심 종목 매수/매도 추천")
    gr.Markdown("투자 목표와 리스크 허용 범위를 입력하고 '기업명'을 입력하면 해당 기업의 분석과 매수/매도 추천을 제공합니다.")
    
    with gr.Row():
        company_name = gr.Textbox(label="기업명", placeholder="예: 삼성전자")
    
    with gr.Row():
        investment_goal = gr.Textbox(label="투자 목표", placeholder="예: 은퇴 자금 마련")
        risk_tolerance = gr.Radio(["낮음", "중간", "높음"], label="리스크 허용 범위", value="중간")
        investment_duration = gr.Slider(1, 30, value=5, label="투자 기간 (년)")
    
    # 기업 분석과 매수/매도 추천 및 최종 결정 결과 표시를 위한 텍스트 박스 컴포넌트 추가
    company_analysis_result = gr.Textbox(label="기업 분석", lines=5, placeholder="여기에 기업 분석이 표시됩니다.")
    trade_recommendation_result = gr.Textbox(label="매수/매도 추천", lines=5, placeholder="여기에 매수/매도 추천이 표시됩니다.")
    final_decision_result = gr.Textbox(label="최종 결정", lines=2, placeholder="최종 매수/매도 결정을 여기에 표시합니다.")

    # 버튼 설정 및 함수 연결
    submit_btn = gr.Button("분석 및 추천 받기")
    submit_btn.click(
        generate_analysis_recommendation_decision,
        inputs=[company_name, investment_goal, risk_tolerance, investment_duration],
        outputs=[company_analysis_result, trade_recommendation_result, final_decision_result]
    )

if __name__ == "__main__":
    file_path = "hana.pdf"
    splits = load_and_split_docs(file_path)
    db_retriever = embed_and_index_docs_with_faiss(splits, "faiss_index")
    demo.launch()

* Running on local URL:  http://127.0.0.1:7895

To create a public link, set `share=True` in `launch()`.


In [16]:
import os
import gradio as gr
from langchain_upstage import (
    ChatUpstage,
    UpstageEmbeddings,
    UpstageDocumentParseLoader,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS  
from langchain.schema import AIMessage, HumanMessage

# 한글 폰트 설정
def set_korean_font():
    if fm.findSystemFonts(fontpaths=None, fontext='ttf'):
        plt.rc('font', family='NanumGothic')
    else:
        print("한글 폰트를 찾을 수 없습니다. 영어 레이블을 사용합니다.")

set_korean_font()

# LLM 인스턴스 생성
def create_llm():
    return ChatUpstage()

# chat 함수 정의
def chat(message, history):
    llm = create_llm()
    response = llm(message)
    return response.content if hasattr(response, 'content') else str(response)

# FAISS 검색을 이용하여 관련 정보 추출 함수
def retrieve_related_info(company_name):
    if db_retriever:
        search_results = db_retriever.get_relevant_documents(company_name)
        return "\n".join([doc.page_content for doc in search_results])
    return "관련 정보를 찾을 수 없습니다."

# 기업 분석 생성 함수
def generate_company_analysis(company_name, goal, risk, duration):
    history = []
    related_info = retrieve_related_info(company_name)

    analysis_message = (
        f"'{company_name}'에 대한 기업 분석을 제공해 주세요. "
        f"투자 목표가 '{goal}', 리스크 허용 범위가 '{risk}', 투자 기간이 {duration}년임을 고려하여, "
        f"다음 관련 정보도 포함해 분석을 제공해 주세요:\n{related_info}"
    )
    analysis = chat(analysis_message, history)
    return analysis.strip()

# 매수/매도 추천 생성 함수 (기업 분석 포함)
def generate_trade_recommendation_with_analysis(company_name, goal, risk, duration):
    history = []
    
    # 기업 분석과 관련 정보 결합
    analysis = generate_company_analysis(company_name, goal, risk, duration)
    related_info = retrieve_related_info(company_name)

    recommendation_message = (
        f"다음은 '{company_name}'에 대한 기업 분석과 관련 정보입니다:\n\n{analysis}\n\n"
        f"투자 목표가 '{goal}', 리스크 허용 범위가 '{risk}', 투자 기간이 {duration}년임을 고려하여, "
        f"이 정보를 바탕으로 매수 또는 매도 추천을 제공해 주세요:\n{related_info}"
    )
    
    recommendation = chat(recommendation_message, history)
    return recommendation.strip()

# 최종 매수/매도 결정 생성 함수
def generate_final_decision(analysis, recommendation):
    history = []
    
    final_decision_message = (
        f"다음은 '{company_name}'에 대한 기업 분석과 매수/매도 추천 내용입니다. "
        f"이 모든 정보를 종합하여 최종적으로 '매수' 또는 '매도' 중 하나를 선택하여 추천해 주세요:\n\n"
        f"기업 분석:\n{analysis}\n\n매수/매도 추천:\n{recommendation}"
    )
    final_decision = chat(final_decision_message, history)
    
    # 최종 결정 추출
    if "매수" in final_decision:
        return "매수"
    elif "매도" in final_decision or "중립" in final_decision or "hold" in final_decision.lower():
        return "매도"
    else:
        return "추천 없음"

# 통합 함수: 기업 분석과 매수/매도 추천 및 최종 결론 출력
def generate_analysis_recommendation_decision(company_name, goal, risk, duration):
    if not company_name:
        return "기업명을 입력해 주세요.", "", ""
    
    # 기업 분석 및 매수/매도 추천 생성
    analysis = generate_company_analysis(company_name, goal, risk, duration)
    recommendation = generate_trade_recommendation_with_analysis(company_name, goal, risk, duration)
    
    # 최종 매수/매도 결론 생성
    final_decision = generate_final_decision(analysis, recommendation)
    
    return analysis, recommendation, final_decision

# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    gr.Markdown("# 관심 종목 매수/매도 추천")
    gr.Markdown("투자 목표와 리스크 허용 범위를 입력하고 '기업명'을 입력하면 해당 기업의 분석과 매수/매도 추천을 제공합니다.")
    
    with gr.Row():
        company_name = gr.Textbox(label="기업명", placeholder="예: 삼성전자")
    
    with gr.Row():
        investment_goal = gr.Textbox(label="투자 목표", placeholder="예: 은퇴 자금 마련")
        risk_tolerance = gr.Radio(["낮음", "중간", "높음"], label="리스크 허용 범위", value="중간")
        investment_duration = gr.Slider(1, 30, value=5, label="투자 기간 (년)")

    # 버튼 설정 및 함수 연결
    submit_btn = gr.Button("분석 및 추천 받기")
    submit_btn.click(
        generate_analysis_recommendation_decision,
        inputs=[company_name, investment_goal, risk_tolerance, investment_duration],
        outputs=[company_analysis_result, trade_recommendation_result, final_decision_result]
    )
    
    final_decision_result = gr.Textbox(label="최종 결정", lines=2, placeholder="최종 매수/매도 결정을 여기에 표시합니다.")

    # 기업 분석과 매수/매도 추천 및 최종 결정 결과 표시를 위한 텍스트 박스 컴포넌트 추가
    company_analysis_result = gr.Textbox(label="기업 분석", lines=5, placeholder="여기에 기업 분석이 표시됩니다.")
    trade_recommendation_result = gr.Textbox(label="매수/매도 추천", lines=5, placeholder="여기에 매수/매도 추천이 표시됩니다.")

if __name__ == "__main__":
    file_path = "hana.pdf"
    splits = load_and_split_docs(file_path)
    db_retriever = embed_and_index_docs_with_faiss(splits, "faiss_index")
    demo.launch()

* Running on local URL:  http://127.0.0.1:7897

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/blocks.py", line 2028, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/blocks.py", line 1832, in postprocess_data
    if block._id in state:
       ^^^^^^^^^^^^^^^^^^
  File

In [17]:
import os
import gradio as gr
from langchain_upstage import (
    ChatUpstage,
    UpstageEmbeddings,
    UpstageDocumentParseLoader,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS  
from langchain.schema import AIMessage, HumanMessage

# 한글 폰트 설정
def set_korean_font():
    if fm.findSystemFonts(fontpaths=None, fontext='ttf'):
        plt.rc('font', family='NanumGothic')
    else:
        print("한글 폰트를 찾을 수 없습니다. 영어 레이블을 사용합니다.")

set_korean_font()

# LLM 인스턴스 생성
def create_llm():
    return ChatUpstage()

# chat 함수 정의
def chat(message, history):
    llm = create_llm()
    response = llm(message)
    return response.content if hasattr(response, 'content') else str(response)

# FAISS 검색을 이용하여 관련 정보 추출 함수
def retrieve_related_info(company_name):
    if db_retriever:
        search_results = db_retriever.get_relevant_documents(company_name)
        return "\n".join([doc.page_content for doc in search_results])
    return "관련 정보를 찾을 수 없습니다."

# 기업 분석 생성 함수
def generate_company_analysis(company_name, goal, risk, duration):
    history = []
    related_info = retrieve_related_info(company_name)

    analysis_message = (
        f"'{company_name}'에 대한 기업 분석을 제공해 주세요. "
        f"투자 목표가 '{goal}', 리스크 허용 범위가 '{risk}', 투자 기간이 {duration}년임을 고려하여, "
        f"다음 관련 정보도 포함해 분석을 제공해 주세요:\n{related_info}"
    )
    analysis = chat(analysis_message, history)
    return analysis.strip()

# 매수/매도 추천 생성 함수 (기업 분석 포함)
def generate_trade_recommendation_with_analysis(company_name, goal, risk, duration):
    history = []
    
    # 기업 분석과 관련 정보 결합
    analysis = generate_company_analysis(company_name, goal, risk, duration)
    related_info = retrieve_related_info(company_name)

    recommendation_message = (
        f"다음은 '{company_name}'에 대한 기업 분석과 관련 정보입니다:\n\n{analysis}\n\n"
        f"투자 목표가 '{goal}', 리스크 허용 범위가 '{risk}', 투자 기간이 {duration}년임을 고려하여, "
        f"이 정보를 바탕으로 매수 또는 매도 추천을 제공해 주세요:\n{related_info}"
    )
    
    recommendation = chat(recommendation_message, history)
    return recommendation.strip()

# 최종 매수/매도 결정 생성 함수
def generate_final_decision(analysis, recommendation):
    history = []
    
    final_decision_message = (
        f"다음은 기업 분석과 매수/매도 추천 내용입니다. "
        f"이 모든 정보를 종합하여 최종적으로 '매수' 또는 '매도' 중 하나를 선택하여 추천해 주세요:\n\n"
        f"기업 분석:\n{analysis}\n\n매수/매도 추천:\n{recommendation}"
    )
    final_decision = chat(final_decision_message, history)
    
    # 최종 결정 추출
    if "매수" in final_decision:
        return "매수"
    elif "매도" in final_decision or "중립" in final_decision or "hold" in final_decision.lower():
        return "매도"
    else:
        return "추천 없음"

# 통합 함수: 기업 분석과 매수/매도 추천 및 최종 결론 출력
def generate_analysis_recommendation_decision(company_name, goal, risk, duration):
    if not company_name:
        return "기업명을 입력해 주세요.", "", ""
    
    # 기업 분석 및 매수/매도 추천 생성
    analysis = generate_company_analysis(company_name, goal, risk, duration)
    recommendation = generate_trade_recommendation_with_analysis(company_name, goal, risk, duration)
    
    # 최종 매수/매도 결론 생성
    final_decision = generate_final_decision(analysis, recommendation)
    
    return analysis, recommendation, final_decision

# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    gr.Markdown("# 관심 종목 매수/매도 추천")
    gr.Markdown("투자 목표와 리스크 허용 범위를 입력하고 '기업명'을 입력하면 해당 기업의 분석과 매수/매도 추천을 제공합니다.")
    
    with gr.Row():
        company_name = gr.Textbox(label="기업명", placeholder="예: 삼성전자")
    
    with gr.Row():
        investment_goal = gr.Textbox(label="투자 목표", placeholder="예: 은퇴 자금 마련")
        risk_tolerance = gr.Radio(["낮음", "중간", "높음"], label="리스크 허용 범위", value="중간")
        investment_duration = gr.Slider(1, 30, value=5, label="투자 기간 (년)")

    # 기업 분석과 매수/매도 추천 및 최종 결정 결과 표시를 위한 텍스트 박스 컴포넌트 추가
    company_analysis_result = gr.Textbox(label="기업 분석", lines=5, placeholder="여기에 기업 분석이 표시됩니다.")
    trade_recommendation_result = gr.Textbox(label="매수/매도 추천", lines=5, placeholder="여기에 매수/매도 추천이 표시됩니다.")
    final_decision_result = gr.Textbox(label="최종 결정", lines=2, placeholder="최종 매수/매도 결정을 여기에 표시합니다.")

    # 버튼 설정 및 함수 연결
    submit_btn = gr.Button("분석 및 추천 받기")
    submit_btn.click(
        generate_analysis_recommendation_decision,
        inputs=[company_name, investment_goal, risk_tolerance, investment_duration],
        outputs=[company_analysis_result, trade_recommendation_result, final_decision_result]
    )

if __name__ == "__main__":
    file_path = "hana.pdf"
    splits = load_and_split_docs(file_path)
    db_retriever = embed_and_index_docs_with_faiss(splits, "faiss_index")
    demo.launch()

* Running on local URL:  http://127.0.0.1:7898

To create a public link, set `share=True` in `launch()`.


In [18]:
import os
import gradio as gr
from langchain_upstage import (
    ChatUpstage,
    UpstageEmbeddings,
    UpstageDocumentParseLoader,
)
from langchain.vectorstores import FAISS

# 한글 폰트 설정
def set_korean_font():
    if fm.findSystemFonts(fontpaths=None, fontext='ttf'):
        plt.rc('font', family='NanumGothic')
    else:
        print("한글 폰트를 찾을 수 없습니다. 영어 레이블을 사용합니다.")

set_korean_font()

# LLM 인스턴스 생성
def create_llm():
    return ChatUpstage()

# chat 함수 정의
def chat(message, history):
    llm = create_llm()
    response = llm(message)
    return response.content if hasattr(response, 'content') else str(response)

# FAISS 검색을 이용하여 관련 정보 추출 함수
def retrieve_related_info(company_name):
    if db_retriever:
        search_results = db_retriever.get_relevant_documents(company_name)
        return "\n".join([doc.page_content for doc in search_results])
    return "관련 정보를 찾을 수 없습니다."

# 기업 분석 생성 함수
def generate_company_analysis(company_name):
    history = []
    related_info = retrieve_related_info(company_name)

    analysis_message = (
        f"'{company_name}'에 대한 기업 분석을 제공해 주세요. "
        f"다음 관련 정보도 포함해 분석을 제공해 주세요:\n{related_info}"
    )
    analysis = chat(analysis_message, history)
    return analysis.strip()

# 매수/매도 추천 생성 함수 (기업 분석 포함)
def generate_trade_recommendation_with_analysis(company_name):
    history = []
    
    # 기업 분석과 관련 정보 결합
    analysis = generate_company_analysis(company_name)
    related_info = retrieve_related_info(company_name)

    recommendation_message = (
        f"다음은 '{company_name}'에 대한 기업 분석과 관련 정보입니다:\n\n{analysis}\n\n"
        f"이 정보를 바탕으로 매수 또는 매도 추천을 제공해 주세요:\n{related_info}"
    )
    
    recommendation = chat(recommendation_message, history)
    return recommendation.strip()

# 최종 매수/매도 결정 생성 함수
def generate_final_decision(analysis, recommendation):
    history = []
    
    final_decision_message = (
        f"다음은 기업 분석과 매수/매도 추천 내용입니다. "
        f"이 모든 정보를 종합하여 최종적으로 '매수' 또는 '매도' 중 하나를 선택하여 추천해 주세요:\n\n"
        f"기업 분석:\n{analysis}\n\n매수/매도 추천:\n{recommendation}"
    )
    final_decision = chat(final_decision_message, history)
    
    # 최종 결정 추출
    if "매수" in final_decision:
        return "매수"
    elif "매도" in final_decision or "중립" in final_decision or "hold" in final_decision.lower():
        return "매도"
    else:
        return "추천 없음"

# 통합 함수: 기업 분석과 매수/매도 추천 및 최종 결론 출력
def generate_analysis_recommendation_decision(company_name):
    if not company_name:
        return "기업명을 입력해 주세요.", "", ""
    
    # 기업 분석 및 매수/매도 추천 생성
    analysis = generate_company_analysis(company_name)
    recommendation = generate_trade_recommendation_with_analysis(company_name)
    
    # 최종 매수/매도 결론 생성
    final_decision = generate_final_decision(analysis, recommendation)
    
    return analysis, recommendation, final_decision

# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    gr.Markdown("# 관심 종목 매수/매도 추천")
    gr.Markdown("기업명을 입력하면 해당 기업의 분석과 매수/매도 추천을 제공합니다.")
    
    with gr.Row():
        company_name = gr.Textbox(label="기업명", placeholder="예: 삼성전자")

    # 기업 분석과 매수/매도 추천 및 최종 결정 결과 표시를 위한 텍스트 박스 컴포넌트 추가
    company_analysis_result = gr.Textbox(label="기업 분석", lines=5, placeholder="여기에 기업 분석이 표시됩니다.")
    trade_recommendation_result = gr.Textbox(label="매수/매도 추천", lines=5, placeholder="여기에 매수/매도 추천이 표시됩니다.")
    final_decision_result = gr.Textbox(label="최종 결정", lines=2, placeholder="최종 매수/매도 결정을 여기에 표시합니다.")

    # 버튼 설정 및 함수 연결
    submit_btn = gr.Button("분석 및 추천 받기")
    submit_btn.click(
        generate_analysis_recommendation_decision,
        inputs=[company_name],
        outputs=[company_analysis_result, trade_recommendation_result, final_decision_result]
    )

if __name__ == "__main__":
    file_path = "hana.pdf"
    splits = load_and_split_docs(file_path)
    db_retriever = embed_and_index_docs_with_faiss(splits, "faiss_index")
    demo.launch()

* Running on local URL:  http://127.0.0.1:7899

To create a public link, set `share=True` in `launch()`.


In [22]:
import os
import gradio as gr
from langchain_upstage import (
    ChatUpstage,
    UpstageEmbeddings,
    UpstageDocumentParseLoader,
)
from langchain.vectorstores import FAISS

# 한글 폰트 설정
def set_korean_font():
    if fm.findSystemFonts(fontpaths=None, fontext='ttf'):
        plt.rc('font', family='NanumGothic')
    else:
        print("한글 폰트를 찾을 수 없습니다. 영어 레이블을 사용합니다.")

set_korean_font()

# LLM 인스턴스 생성
def create_llm():
    return ChatUpstage()

# chat 함수 정의
def chat(message, history):
    llm = create_llm()
    response = llm(message)
    return response.content if hasattr(response, 'content') else str(response)

# FAISS 검색을 이용하여 관련 정보 추출 함수
def retrieve_related_info(company_name):
    if db_retriever:
        search_results = db_retriever.get_relevant_documents(company_name)
        return "\n".join([doc.page_content for doc in search_results])
    return "관련 정보를 찾을 수 없습니다."

# 기업 분석 생성 함수
def generate_company_analysis(company_name):
    history = []
    related_info = retrieve_related_info(company_name)

    analysis_message = (
        f"'{company_name}'에 대한 기업 분석을 제공해 주세요. "
        f"다음 관련 정보도 포함해 분석을 제공해 주세요:\n{related_info}"
    )
    analysis = chat(analysis_message, history)
    return analysis.strip()

# 매수/매도 추천 생성 함수 (기업 분석 포함)
def generate_trade_recommendation_with_analysis(company_name):
    history = []
    
    # 기업 분석과 관련 정보 결합
    analysis = generate_company_analysis(company_name)
    related_info = retrieve_related_info(company_name)

    recommendation_message = (
        f"다음은 '{company_name}'에 대한 기업 분석과 관련 정보입니다:\n\n{analysis}\n\n"
        f"이 정보를 바탕으로 매수 또는 매도 추천을 제공해 주세요:\n{related_info}"
    )
    
    recommendation = chat(recommendation_message, history)
    return recommendation.strip()

# 최종 매수/매도 결정 생성 함수
def generate_final_decision(analysis, recommendation):
    history = []
    
    final_decision_message = (
        f"다음은 기업 분석과 매수/매도 추천 내용입니다. "
        f"이 모든 정보를 종합하여 최종적으로 '매수' 또는 '매도' 중 하나를 선택하여 추천해 주세요:\n\n"
        f"기업 분석:\n{analysis}\n\n매수/매도 추천:\n{recommendation}"
    )
    final_decision = chat(final_decision_message, history)
    
    # 최종 결정 추출
    if "매수" in final_decision:
        return "매수"
    elif "매도" in final_decision or "중립" in final_decision or "hold" in final_decision.lower():
        return "매도"
    else:
        return "추천 없음"

# 통합 함수: 기업 분석과 매수/매도 추천 및 최종 결론 출력
def generate_analysis_recommendation_decision(company_name):
    if not company_name:
        return "기업명을 입력해 주세요.", "", ""
    
    # 기업 분석 및 매수/매도 추천 생성
    analysis = generate_company_analysis(company_name)
    recommendation = generate_trade_recommendation_with_analysis(company_name)
    
    # 최종 매수/매도 결론 생성
    final_decision = generate_final_decision(analysis, recommendation)
    
    return analysis, recommendation, final_decision

# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    gr.Markdown("# 관심 종목 매수/매도 추천")
    gr.Markdown("기업명을 입력하면 해당 기업의 분석과 매수/매도 추천을 제공합니다.")
    
    with gr.Row():
        company_name = gr.Textbox(label="기업명", placeholder="예: 삼성전자")

    # 기업명 아래에 버튼 추가
    submit_btn = gr.Button("분석 및 추천 받기")

    # 기업 분석과 매수/매도 추천 및 최종 결정 결과 표시를 위한 텍스트 박스 컴포넌트 추가
    company_analysis_result = gr.Textbox(label="기업 분석", lines=5, placeholder="여기에 기업 분석이 표시됩니다.")
    trade_recommendation_result = gr.Textbox(label="매수/매도 추천", lines=5, placeholder="여기에 매수/매도 추천이 표시됩니다.")
    final_decision_result = gr.Textbox(label="최종 결정", lines=2, placeholder="최종 매수/매도 결정을 여기에 표시합니다.")

    # 버튼 클릭 이벤트 설정 및 연결
    submit_btn.click(
        generate_analysis_recommendation_decision,
        inputs=[company_name],
        outputs=[company_analysis_result, trade_recommendation_result, final_decision_result]
    )

if __name__ == "__main__":
    file_path = "hana.pdf"
    splits = load_and_split_docs(file_path)
    db_retriever = embed_and_index_docs_with_faiss(splits, "faiss_index")
    demo.launch()

* Running on local URL:  http://127.0.0.1:7900

To create a public link, set `share=True` in `launch()`.


In [23]:
import os
import gradio as gr
from langchain_upstage import (
    ChatUpstage,
    UpstageEmbeddings,
    UpstageDocumentParseLoader,
)
from langchain.vectorstores import FAISS

# 한글 폰트 설정
def set_korean_font():
    if fm.findSystemFonts(fontpaths=None, fontext='ttf'):
        plt.rc('font', family='NanumGothic')
    else:
        print("한글 폰트를 찾을 수 없습니다. 영어 레이블을 사용합니다.")

set_korean_font()

# LLM 인스턴스 생성
def create_llm():
    return ChatUpstage()

# chat 함수 정의
def chat(message, history):
    llm = create_llm()
    response = llm(message)
    return response.content if hasattr(response, 'content') else str(response)

# 산업군 추출 함수
def extract_industry(company_name):
    industry_prompt = f"'{company_name}'의 산업군을 알려주세요."
    return chat(industry_prompt, []).strip()

# FAISS 검색을 이용하여 관련 정보 추출 함수
def retrieve_related_info(company_name):
    # 기업명으로 산업군 추출
    industry = extract_industry(company_name)
    combined_query = f"{company_name} {industry}"
    
    if db_retriever:
        search_results = db_retriever.get_relevant_documents(combined_query)
        return "\n".join([doc.page_content for doc in search_results])
    return "관련 정보를 찾을 수 없습니다."

# 기업 분석 생성 함수
def generate_company_analysis(company_name):
    history = []
    related_info = retrieve_related_info(company_name)

    analysis_message = (
        f"'{company_name}'에 대한 기업 분석을 제공해 주세요. "
        f"다음 관련 정보도 포함해 분석을 제공해 주세요:\n{related_info}"
    )
    analysis = chat(analysis_message, history)
    return analysis.strip()

# 매수/매도 추천 생성 함수 (기업 분석 포함)
def generate_trade_recommendation_with_analysis(company_name):
    history = []
    
    # 기업 분석과 관련 정보 결합
    analysis = generate_company_analysis(company_name)
    related_info = retrieve_related_info(company_name)

    recommendation_message = (
        f"다음은 '{company_name}'에 대한 기업 분석과 관련 정보입니다:\n\n{analysis}\n\n"
        f"이 정보를 바탕으로 매수 또는 매도 추천을 제공해 주세요:\n{related_info}"
    )
    
    recommendation = chat(recommendation_message, history)
    return recommendation.strip()

# 최종 매수/매도 결정 생성 함수
def generate_final_decision(analysis, recommendation):
    history = []
    
    final_decision_message = (
        f"다음은 기업 분석과 매수/매도 추천 내용입니다. "
        f"이 모든 정보를 종합하여 최종적으로 '매수' 또는 '매도' 중 하나를 선택하여 추천해 주세요:\n\n"
        f"기업 분석:\n{analysis}\n\n매수/매도 추천:\n{recommendation}"
    )
    final_decision = chat(final_decision_message, history)
    
    # 최종 결정 추출
    if "매수" in final_decision:
        return "매수"
    elif "매도" in final_decision or "중립" in final_decision or "hold" in final_decision.lower():
        return "매도"
    else:
        return "추천 없음"

# 통합 함수: 기업 분석과 매수/매도 추천 및 최종 결론 출력
def generate_analysis_recommendation_decision(company_name):
    if not company_name:
        return "기업명을 입력해 주세요.", "", ""
    
    # 기업 분석 및 매수/매도 추천 생성
    analysis = generate_company_analysis(company_name)
    recommendation = generate_trade_recommendation_with_analysis(company_name)
    
    # 최종 매수/매도 결론 생성
    final_decision = generate_final_decision(analysis, recommendation)
    
    return analysis, recommendation, final_decision

# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    gr.Markdown("# 관심 종목 매수/매도 추천")
    gr.Markdown("기업명을 입력하면 해당 기업의 분석과 매수/매도 추천을 제공합니다.")
    
    with gr.Row():
        company_name = gr.Textbox(label="기업명", placeholder="예: 삼성전자")

    # 기업명 아래에 버튼 추가
    submit_btn = gr.Button("분석 및 추천 받기")

    # 기업 분석과 매수/매도 추천 및 최종 결정 결과 표시를 위한 텍스트 박스 컴포넌트 추가
    company_analysis_result = gr.Textbox(label="기업 분석", lines=5, placeholder="여기에 기업 분석이 표시됩니다.")
    trade_recommendation_result = gr.Textbox(label="매수/매도 추천", lines=5, placeholder="여기에 매수/매도 추천이 표시됩니다.")
    final_decision_result = gr.Textbox(label="최종 결정", lines=2, placeholder="최종 매수/매도 결정을 여기에 표시합니다.")

    # 버튼 클릭 이벤트 설정 및 연결
    submit_btn.click(
        generate_analysis_recommendation_decision,
        inputs=[company_name],
        outputs=[company_analysis_result, trade_recommendation_result, final_decision_result]
    )

if __name__ == "__main__":
    file_path = "hana.pdf"
    splits = load_and_split_docs(file_path)
    db_retriever = embed_and_index_docs_with_faiss(splits, "faiss_index")
    demo.launch()

* Running on local URL:  http://127.0.0.1:7901

To create a public link, set `share=True` in `launch()`.


In [24]:
import os
import gradio as gr
import sqlite3
from langchain_upstage import (
    ChatUpstage,
    UpstageEmbeddings,
    UpstageDocumentParseLoader,
)
from langchain.vectorstores import FAISS

# 데이터베이스에서 종목 정보를 가져오는 함수
def fetch_stock_info(company_name):
    conn = sqlite3.connect("info_stock.db")
    cursor = conn.cursor()
    
    query = "SELECT * FROM stocks WHERE company_name = ?"
    cursor.execute(query, (company_name,))
    result = cursor.fetchone()
    
    conn.close()
    
    if result:
        # 데이터 형식에 맞춰 정보 추출 (예시)
        columns = ["company_name", "industry", "market_cap", "p_e_ratio", "dividend_yield"]
        stock_info = {columns[i]: result[i] for i in range(len(columns))}
        return stock_info
    else:
        return None

# LLM 인스턴스 생성
def create_llm():
    return ChatUpstage()

# chat 함수 정의
def chat(message, history):
    llm = create_llm()
    response = llm(message)
    return response.content if hasattr(response, 'content') else str(response)

# FAISS 검색을 이용하여 관련 정보 추출 함수
def retrieve_related_info(company_name):
    # 데이터베이스에서 기업 정보 가져오기
    stock_info = fetch_stock_info(company_name)
    if stock_info:
        industry = stock_info.get("industry", "")
        combined_query = f"{company_name} {industry}"
    else:
        return "관련 정보를 찾을 수 없습니다."

    # FAISS 검색
    if db_retriever:
        search_results = db_retriever.get_relevant_documents(combined_query)
        return "\n".join([doc.page_content for doc in search_results])
    return "관련 정보를 찾을 수 없습니다."

# 기업 분석 생성 함수
def generate_company_analysis(company_name):
    history = []
    related_info = retrieve_related_info(company_name)
    
    # 데이터베이스에서 기업 정보 가져오기
    stock_info = fetch_stock_info(company_name)
    if stock_info:
        stock_info_text = (
            f"기업명: {stock_info['company_name']}\n"
            f"산업군: {stock_info['industry']}\n"
            f"시가총액: {stock_info['market_cap']}\n"
            f"PER: {stock_info['p_e_ratio']}\n"
            f"배당수익률: {stock_info['dividend_yield']}\n"
        )
    else:
        stock_info_text = "DB에서 기업 정보를 찾을 수 없습니다."

    analysis_message = (
        f"'{company_name}'에 대한 기업 분석을 제공해 주세요. "
        f"다음 관련 정보도 포함해 분석을 제공해 주세요:\n{related_info}\n\n"
        f"데이터베이스 정보:\n{stock_info_text}"
    )
    analysis = chat(analysis_message, history)
    return analysis.strip()

# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    gr.Markdown("# 관심 종목 매수/매도 추천")
    gr.Markdown("기업명을 입력하면 해당 기업의 분석과 매수/매도 추천을 제공합니다.")
    
    with gr.Row():
        company_name = gr.Textbox(label="기업명", placeholder="예: 삼성전자")

    # 기업명 아래에 버튼 추가
    submit_btn = gr.Button("분석 및 추천 받기")

    # 기업 분석과 매수/매도 추천 및 최종 결정 결과 표시를 위한 텍스트 박스 컴포넌트 추가
    company_analysis_result = gr.Textbox(label="기업 분석", lines=5, placeholder="여기에 기업 분석이 표시됩니다.")
    trade_recommendation_result = gr.Textbox(label="매수/매도 추천", lines=5, placeholder="여기에 매수/매도 추천이 표시됩니다.")
    final_decision_result = gr.Textbox(label="최종 결정", lines=2, placeholder="최종 매수/매도 결정을 여기에 표시합니다.")

    # 버튼 클릭 이벤트 설정 및 연결
    submit_btn.click(
        generate_analysis_recommendation_decision,
        inputs=[company_name],
        outputs=[company_analysis_result, trade_recommendation_result, final_decision_result]
    )

if __name__ == "__main__":
    file_path = "hana.pdf"
    splits = load_and_split_docs(file_path)
    db_retriever = embed_and_index_docs_with_faiss(splits, "faiss_index")
    demo.launch()

* Running on local URL:  http://127.0.0.1:7916

To create a public link, set `share=True` in `launch()`.


In [26]:
import os
import gradio as gr
import sqlite3
from langchain_upstage import (
    ChatUpstage,
    UpstageEmbeddings,
    UpstageDocumentParseLoader,
)
from langchain.vectorstores import FAISS

# 데이터베이스에서 종목 정보를 가져오는 함수
def fetch_stock_info(company_name):
    conn = sqlite3.connect("info_stock.db")
    cursor = conn.cursor()
    
    query = "SELECT * FROM stock_data WHERE 종목명 = ?"
    cursor.execute(query, (company_name,))
    result = cursor.fetchone()
    
    conn.close()
    
    if result:
        # 데이터 형식에 맞춰 정보 추출 (예시)
        columns = ["종목코드", "종목명", "주당액면가액(원)", "(연결)당기순이익(백만원)", "(별도)당기순이익(백만원)",
                   "(연결)주당순이익(원)", "현금배당금총액(백만원)", "당기금액", "계정명", "당기명", "thstrm_dt",
                   "개괄", "투자 의견", "현재가", "Close", "%Change", "Market cap."]
        stock_info = {columns[i]: result[i] for i in range(len(columns))}
        return stock_info
    else:
        return None

# LLM 인스턴스 생성
def create_llm():
    return ChatUpstage()

# chat 함수 정의
def chat(message, history):
    llm = create_llm()
    response = llm(message)
    return response.content if hasattr(response, 'content') else str(response)

# FAISS 검색을 이용하여 관련 정보 추출 함수
def retrieve_related_info(company_name):
    # 데이터베이스에서 기업 정보 가져오기
    stock_info = fetch_stock_info(company_name)
    if stock_info:
        combined_query = f"{company_name} {stock_info.get('개괄', '')}"
    else:
        return "관련 정보를 찾을 수 없습니다."

    # FAISS 검색
    if db_retriever:
        search_results = db_retriever.get_relevant_documents(combined_query)
        return "\n".join([doc.page_content for doc in search_results])
    return "관련 정보를 찾을 수 없습니다."

# 기업 분석 생성 함수
def generate_company_analysis(company_name):
    history = []
    related_info = retrieve_related_info(company_name)
    
    # 데이터베이스에서 기업 정보 가져오기
    stock_info = fetch_stock_info(company_name)
    if stock_info:
        stock_info_text = (
            f"종목명: {stock_info['종목명']}\n"
            f"종목코드: {stock_info['종목코드']}\n"
            f"현재가: {stock_info['현재가']}원\n"
            f"시가총액: {stock_info['Market cap.']}\n"
            f"PER: {stock_info['(연결)주당순이익(원)']}\n"
            f"배당수익률: {stock_info['현금배당금총액(백만원)']}\n"
            f"투자 의견: {stock_info['투자 의견']}\n"
            f"개괄: {stock_info['개괄']}\n"
        )
    else:
        stock_info_text = "DB에서 기업 정보를 찾을 수 없습니다."

    analysis_message = (
        f"'{company_name}'에 대한 기업 분석을 제공해 주세요. "
        f"다음 관련 정보도 포함해 분석을 제공해 주세요:\n{related_info}\n\n"
        f"데이터베이스 정보:\n{stock_info_text}"
    )
    analysis = chat(analysis_message, history)
    return analysis.strip()

# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    gr.Markdown("# 관심 종목 매수/매도 추천")
    gr.Markdown("기업명을 입력하면 해당 기업의 분석과 매수/매도 추천을 제공합니다.")
    
    with gr.Row():
        company_name = gr.Textbox(label="기업명", placeholder="예: 삼성전자")

    submit_btn = gr.Button("분석 및 추천 받기")

    # 기업 분석과 매수/매도 추천 및 최종 결정 결과 표시를 위한 텍스트 박스 컴포넌트 추가
    company_analysis_result = gr.Textbox(label="기업 분석", lines=5, placeholder="여기에 기업 분석이 표시됩니다.")
    trade_recommendation_result = gr.Textbox(label="매수/매도 추천", lines=5, placeholder="여기에 매수/매도 추천이 표시됩니다.")
    final_decision_result = gr.Textbox(label="최종 결정", lines=2, placeholder="최종 매수/매도 결정을 여기에 표시합니다.")

    submit_btn.click(
        generate_analysis_recommendation_decision,
        inputs=[company_name],
        outputs=[company_analysis_result, trade_recommendation_result, final_decision_result]
    )

if __name__ == "__main__":
    file_path = "hana.pdf"
    splits = load_and_split_docs(file_path)
    db_retriever = embed_and_index_docs_with_faiss(splits, "faiss_index")
    demo.launch()

* Running on local URL:  http://127.0.0.1:7918

To create a public link, set `share=True` in `launch()`.
